## Bactesting bollinger band 

In [ ]:
import sys
import os

# Add the parent directory (d:\Trading bot) to the Python path
sys.path.append(os.path.abspath("D:/Trading bot/"))

In [12]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display
from backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [15]:
symbol = 'USDJPY'
start_dt = datetime(2023, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)

sma_period = 20 

ohlc['sma_20'] = ohlc['open'].rolling(sma_period).mean()
ohlc['std'] = ohlc['open'].rolling(sma_period).std()

ohlc['upper_band'] = ohlc['sma_20'] + 1.5 * ohlc['std']
ohlc['lower_band'] = ohlc['sma_20'] - 1.5 * ohlc['std']

display(ohlc)

fig = create_price_fig(ohlc, indicators=['sma_20' ,'upper_band', 'lower_band'])
display(fig)

,time,open,high,low,close,sma_20,std,upper_band,lower_band
0,2023-01-02,130.924,131.046,130.601,130.601,NaN,NaN,NaN,NaN
1,2023-01-03,130.698,131.400,129.517,130.985,NaN,NaN,NaN,NaN
2,2023-01-04,131.014,132.716,129.926,132.652,NaN,NaN,NaN,NaN
3,2023-01-05,132.570,134.055,131.684,133.421,NaN,NaN,NaN,NaN
4,2023-01-06,133.406,134.774,131.998,132.106,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
661,2025-07-21,147.844,148.667,147.081,147.334,145.90795,1.678171,148.425207,143.390693
662,2025-07-22,147.330,147.949,146.310,146.637,145.96860,1.707782,148.530273,143.406927
663,2025-07-23,146.590,147.212,146.109,146.473,146.05320,1.693807,148.593911,143.512489
664,2025-07-24,146.472,147.019,145.856,146.998,146.11675,1.683966,148.642698,143.590802


In [28]:
def get_signal(x):
    if x['open'] > x['upper_band']:
        return 'sell'
    elif x['open'] < x['lower_band']:
        return 'buy'
    

def exit_singnal(x):
    if x['open'] > x['sma_20']:
        return 'close_buy_postion'
    elif x['open'] < x['lower_band']:
        return 'close_sell_postion'
    

ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc['exit_signal'] = ohlc.apply(exit_singnal, axis=1)
ohlc

,time,open,high,low,close,sma_20,std,upper_band,lower_band,signal,exit_signal
0,2023-01-02,130.924,131.046,130.601,130.601,NaN,NaN,NaN,NaN,None,None
1,2023-01-03,130.698,131.400,129.517,130.985,NaN,NaN,NaN,NaN,None,None
2,2023-01-04,131.014,132.716,129.926,132.652,NaN,NaN,NaN,NaN,None,None
3,2023-01-05,132.570,134.055,131.684,133.421,NaN,NaN,NaN,NaN,None,None
4,2023-01-06,133.406,134.774,131.998,132.106,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...
661,2025-07-21,147.844,148.667,147.081,147.334,145.90795,1.678171,148.425207,143.390693,None,close_buy_postion
662,2025-07-22,147.330,147.949,146.310,146.637,145.96860,1.707782,148.530273,143.406927,None,close_buy_postion
663,2025-07-23,146.590,147.212,146.109,146.473,146.05320,1.693807,148.593911,143.512489,None,close_buy_postion
664,2025-07-24,146.472,147.019,145.856,146.998,146.11675,1.683966,148.642698,143.590802,None,close_buy_postion


In [29]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots 
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal 
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data['exit_signal'] == 'close_buy_postion':
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data['exit_signal'] == 'close_sell_postion':
            orders.close_trade(trade)

In [30]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1/150
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,USDJPY,sell,100000,2023-02-06 00:00:00,132.448,2023-03-14 00:00:00,133.176,0,0,{},-485.33,-7.0,-492.33,-492.33,9507.67
1,closed,USDJPY,buy,100000,2023-03-16 00:00:00,133.373,2023-04-03 00:00:00,133.367,0,0,{},-4.00,-7.0,-11.0,-503.33,9496.67
2,closed,USDJPY,sell,100000,2023-04-17 00:00:00,133.802,2023-07-12 00:00:00,140.376,0,0,{},-4382.67,-7.0,-4389.67,-4893.0,5107.0
3,closed,USDJPY,buy,100000,2023-07-13 00:00:00,138.456,2023-07-31 00:00:00,140.918,0,0,{},1641.33,-7.0,1634.33,-3258.67,6741.33
4,closed,USDJPY,sell,100000,2023-08-03 00:00:00,143.324,2023-11-21 00:00:00,148.323,0,0,{},-3332.67,-7.0,-3339.67,-6598.34,3401.66
5,closed,USDJPY,buy,100000,2023-11-22 00:00:00,148.372,2024-01-05 00:00:00,144.604,0,0,{},-2512.00,-7.0,-2519.0,-9117.34,882.66
6,closed,USDJPY,sell,100000,2024-01-11 00:00:00,145.74,2024-03-07 00:00:00,149.306,0,0,{},-2377.33,-7.0,-2384.33,-11501.67,-1501.67
7,closed,USDJPY,buy,100000,2024-03-08 00:00:00,147.96,2024-03-20 00:00:00,150.809,0,0,{},1899.33,-7.0,1892.33,-9609.34,390.66
8,closed,USDJPY,sell,100000,2024-04-11 00:00:00,153.079,2024-06-05 00:00:00,154.849,0,0,{},-1180.00,-7.0,-1187.0,-10796.34,-796.34
9,closed,USDJPY,sell,100000,2024-06-20 00:00:00,158.08,2024-07-18 00:00:00,156.165,0,0,{},1276.67,-7.0,1269.67,-9526.67,473.33


In [31]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [33]:
backtest_fig = bt.visualize_backtest(indicators=['sma_20', 'upper_band', 'lower_band'])
backtest_fig

In [34]:
evaluate_backtest(bt.trades)

biggest_profit 3092.0
biggest_loss -7570.67


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
3,closed,USDJPY,buy,100000,2023-07-13,138.456,2023-07-31,140.918,0,0,{},1641.33,-7.0,1634.33,-3258.67,6741.33
7,closed,USDJPY,buy,100000,2024-03-08,147.96,2024-03-20,150.809,0,0,{},1899.33,-7.0,1892.33,-9609.34,390.66
9,closed,USDJPY,sell,100000,2024-06-20,158.08,2024-07-18,156.165,0,0,{},1276.67,-7.0,1269.67,-9526.67,473.33
11,closed,USDJPY,buy,100000,2024-09-05,143.728,2024-09-23,143.838,0,0,{},73.33,-7.0,66.33,-17038.01,-7038.01
13,closed,USDJPY,buy,100000,2024-12-02,149.58,2024-12-12,152.44,0,0,{},1906.67,-7.0,1899.67,-18214.01,-8214.01
14,closed,USDJPY,sell,100000,2024-12-20,157.416,2025-01-17,155.142,0,0,{},1516.00,-7.0,1509.0,-16705.01,-6705.01
17,closed,USDJPY,sell,100000,2025-03-25,150.691,2025-04-04,146.053,0,0,{},3092.00,-7.0,3085.0,-14683.34,-4683.34
18,closed,USDJPY,buy,100000,2025-04-07,145.34,2025-05-02,145.383,0,0,{},28.67,-7.0,21.67,-14661.67,-4661.67


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,USDJPY,sell,100000,2023-02-06,132.448,2023-03-14,133.176,0,0,{},-485.33,-7.0,-492.33,-492.33,9507.67
1,closed,USDJPY,buy,100000,2023-03-16,133.373,2023-04-03,133.367,0,0,{},-4.00,-7.0,-11.0,-503.33,9496.67
2,closed,USDJPY,sell,100000,2023-04-17,133.802,2023-07-12,140.376,0,0,{},-4382.67,-7.0,-4389.67,-4893.0,5107.0
4,closed,USDJPY,sell,100000,2023-08-03,143.324,2023-11-21,148.323,0,0,{},-3332.67,-7.0,-3339.67,-6598.34,3401.66
5,closed,USDJPY,buy,100000,2023-11-22,148.372,2024-01-05,144.604,0,0,{},-2512.00,-7.0,-2519.0,-9117.34,882.66
6,closed,USDJPY,sell,100000,2024-01-11,145.74,2024-03-07,149.306,0,0,{},-2377.33,-7.0,-2384.33,-11501.67,-1501.67
8,closed,USDJPY,sell,100000,2024-04-11,153.079,2024-06-05,154.849,0,0,{},-1180.00,-7.0,-1187.0,-10796.34,-796.34
10,closed,USDJPY,buy,100000,2024-07-19,157.354,2024-09-02,145.998,0,0,{},-7570.67,-7.0,-7577.67,-17104.34,-7104.34
12,closed,USDJPY,sell,100000,2024-10-03,146.455,2024-11-28,151.058,0,0,{},-3068.67,-7.0,-3075.67,-20113.68,-10113.68
15,closed,USDJPY,buy,100000,2025-01-21,155.563,2025-02-13,154.409,0,0,{},-769.33,-7.0,-776.33,-17481.34,-7481.34


avg_win 1429.25
avg_loss -2238.1666666666665
count_profit_trades 8
count_loss_trades 12
win_rate 0.67
rrr 0.64


,order_type,profit
0,buy,-5586.67
1,sell,-9837.33


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek
0,closed,USDJPY,sell,100000,2023-02-06,132.448,2023-03-14,133.176,0,0,{},-485.33,-7.0,-492.33,-492.33,9507.67,1
1,closed,USDJPY,buy,100000,2023-03-16,133.373,2023-04-03,133.367,0,0,{},-4.00,-7.0,-11.0,-503.33,9496.67,0
2,closed,USDJPY,sell,100000,2023-04-17,133.802,2023-07-12,140.376,0,0,{},-4382.67,-7.0,-4389.67,-4893.0,5107.0,2
3,closed,USDJPY,buy,100000,2023-07-13,138.456,2023-07-31,140.918,0,0,{},1641.33,-7.0,1634.33,-3258.67,6741.33,0
4,closed,USDJPY,sell,100000,2023-08-03,143.324,2023-11-21,148.323,0,0,{},-3332.67,-7.0,-3339.67,-6598.34,3401.66,1
5,closed,USDJPY,buy,100000,2023-11-22,148.372,2024-01-05,144.604,0,0,{},-2512.00,-7.0,-2519.0,-9117.34,882.66,4
6,closed,USDJPY,sell,100000,2024-01-11,145.74,2024-03-07,149.306,0,0,{},-2377.33,-7.0,-2384.33,-11501.67,-1501.67,3
7,closed,USDJPY,buy,100000,2024-03-08,147.96,2024-03-20,150.809,0,0,{},1899.33,-7.0,1892.33,-9609.34,390.66,2
8,closed,USDJPY,sell,100000,2024-04-11,153.079,2024-06-05,154.849,0,0,{},-1180.00,-7.0,-1187.0,-10796.34,-796.34,2
9,closed,USDJPY,sell,100000,2024-06-20,158.08,2024-07-18,156.165,0,0,{},1276.67,-7.0,1269.67,-9526.67,473.33,3


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek,current_max,drawdown
0,closed,USDJPY,sell,100000,2023-02-06,132.448,2023-03-14,133.176,0,0,{},-485.33,-7.0,-492.33,-492.33,9507.67,1,-492.33,0.0
1,closed,USDJPY,buy,100000,2023-03-16,133.373,2023-04-03,133.367,0,0,{},-4.00,-7.0,-11.0,-503.33,9496.67,0,-492.33,-11.0
2,closed,USDJPY,sell,100000,2023-04-17,133.802,2023-07-12,140.376,0,0,{},-4382.67,-7.0,-4389.67,-4893.0,5107.0,2,-492.33,-4400.67
3,closed,USDJPY,buy,100000,2023-07-13,138.456,2023-07-31,140.918,0,0,{},1641.33,-7.0,1634.33,-3258.67,6741.33,0,-492.33,-2766.34
4,closed,USDJPY,sell,100000,2023-08-03,143.324,2023-11-21,148.323,0,0,{},-3332.67,-7.0,-3339.67,-6598.34,3401.66,1,-492.33,-6106.01
5,closed,USDJPY,buy,100000,2023-11-22,148.372,2024-01-05,144.604,0,0,{},-2512.00,-7.0,-2519.0,-9117.34,882.66,4,-492.33,-8625.01
6,closed,USDJPY,sell,100000,2024-01-11,145.74,2024-03-07,149.306,0,0,{},-2377.33,-7.0,-2384.33,-11501.67,-1501.67,3,-492.33,-11009.34
7,closed,USDJPY,buy,100000,2024-03-08,147.96,2024-03-20,150.809,0,0,{},1899.33,-7.0,1892.33,-9609.34,390.66,2,-492.33,-9117.01
8,closed,USDJPY,sell,100000,2024-04-11,153.079,2024-06-05,154.849,0,0,{},-1180.00,-7.0,-1187.0,-10796.34,-796.34,2,-492.33,-10304.01
9,closed,USDJPY,sell,100000,2024-06-20,158.08,2024-07-18,156.165,0,0,{},1276.67,-7.0,1269.67,-9526.67,473.33,3,-492.33,-9034.34


ValueError: Plotly Express cannot process wide-form data with columns of different type.

In [35]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [36]:
evaluate_backtest(bt.trades)


biggest_profit 3092.0
biggest_loss -7570.67


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
3,closed,USDJPY,buy,100000,2023-07-13,138.456,2023-07-31,140.918,0,0,{},1641.33,-7.0,1634.33,-3258.67,6741.33
7,closed,USDJPY,buy,100000,2024-03-08,147.96,2024-03-20,150.809,0,0,{},1899.33,-7.0,1892.33,-9609.34,390.66
9,closed,USDJPY,sell,100000,2024-06-20,158.08,2024-07-18,156.165,0,0,{},1276.67,-7.0,1269.67,-9526.67,473.33
11,closed,USDJPY,buy,100000,2024-09-05,143.728,2024-09-23,143.838,0,0,{},73.33,-7.0,66.33,-17038.01,-7038.01
13,closed,USDJPY,buy,100000,2024-12-02,149.58,2024-12-12,152.44,0,0,{},1906.67,-7.0,1899.67,-18214.01,-8214.01
14,closed,USDJPY,sell,100000,2024-12-20,157.416,2025-01-17,155.142,0,0,{},1516.00,-7.0,1509.0,-16705.01,-6705.01
17,closed,USDJPY,sell,100000,2025-03-25,150.691,2025-04-04,146.053,0,0,{},3092.00,-7.0,3085.0,-14683.34,-4683.34
18,closed,USDJPY,buy,100000,2025-04-07,145.34,2025-05-02,145.383,0,0,{},28.67,-7.0,21.67,-14661.67,-4661.67


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,USDJPY,sell,100000,2023-02-06,132.448,2023-03-14,133.176,0,0,{},-485.33,-7.0,-492.33,-492.33,9507.67
1,closed,USDJPY,buy,100000,2023-03-16,133.373,2023-04-03,133.367,0,0,{},-4.00,-7.0,-11.0,-503.33,9496.67
2,closed,USDJPY,sell,100000,2023-04-17,133.802,2023-07-12,140.376,0,0,{},-4382.67,-7.0,-4389.67,-4893.0,5107.0
4,closed,USDJPY,sell,100000,2023-08-03,143.324,2023-11-21,148.323,0,0,{},-3332.67,-7.0,-3339.67,-6598.34,3401.66
5,closed,USDJPY,buy,100000,2023-11-22,148.372,2024-01-05,144.604,0,0,{},-2512.00,-7.0,-2519.0,-9117.34,882.66
6,closed,USDJPY,sell,100000,2024-01-11,145.74,2024-03-07,149.306,0,0,{},-2377.33,-7.0,-2384.33,-11501.67,-1501.67
8,closed,USDJPY,sell,100000,2024-04-11,153.079,2024-06-05,154.849,0,0,{},-1180.00,-7.0,-1187.0,-10796.34,-796.34
10,closed,USDJPY,buy,100000,2024-07-19,157.354,2024-09-02,145.998,0,0,{},-7570.67,-7.0,-7577.67,-17104.34,-7104.34
12,closed,USDJPY,sell,100000,2024-10-03,146.455,2024-11-28,151.058,0,0,{},-3068.67,-7.0,-3075.67,-20113.68,-10113.68
15,closed,USDJPY,buy,100000,2025-01-21,155.563,2025-02-13,154.409,0,0,{},-769.33,-7.0,-776.33,-17481.34,-7481.34


avg_win 1429.25
avg_loss -2238.1666666666665
count_profit_trades 8
count_loss_trades 12
win_rate 0.67
rrr 0.64


,order_type,profit
0,buy,-5586.67
1,sell,-9837.33


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek
0,closed,USDJPY,sell,100000,2023-02-06,132.448,2023-03-14,133.176,0,0,{},-485.33,-7.0,-492.33,-492.33,9507.67,1
1,closed,USDJPY,buy,100000,2023-03-16,133.373,2023-04-03,133.367,0,0,{},-4.00,-7.0,-11.0,-503.33,9496.67,0
2,closed,USDJPY,sell,100000,2023-04-17,133.802,2023-07-12,140.376,0,0,{},-4382.67,-7.0,-4389.67,-4893.0,5107.0,2
3,closed,USDJPY,buy,100000,2023-07-13,138.456,2023-07-31,140.918,0,0,{},1641.33,-7.0,1634.33,-3258.67,6741.33,0
4,closed,USDJPY,sell,100000,2023-08-03,143.324,2023-11-21,148.323,0,0,{},-3332.67,-7.0,-3339.67,-6598.34,3401.66,1
5,closed,USDJPY,buy,100000,2023-11-22,148.372,2024-01-05,144.604,0,0,{},-2512.00,-7.0,-2519.0,-9117.34,882.66,4
6,closed,USDJPY,sell,100000,2024-01-11,145.74,2024-03-07,149.306,0,0,{},-2377.33,-7.0,-2384.33,-11501.67,-1501.67,3
7,closed,USDJPY,buy,100000,2024-03-08,147.96,2024-03-20,150.809,0,0,{},1899.33,-7.0,1892.33,-9609.34,390.66,2
8,closed,USDJPY,sell,100000,2024-04-11,153.079,2024-06-05,154.849,0,0,{},-1180.00,-7.0,-1187.0,-10796.34,-796.34,2
9,closed,USDJPY,sell,100000,2024-06-20,158.08,2024-07-18,156.165,0,0,{},1276.67,-7.0,1269.67,-9526.67,473.33,3


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek,current_max,drawdown
0,closed,USDJPY,sell,100000,2023-02-06,132.448,2023-03-14,133.176,0,0,{},-485.33,-7.0,-492.33,-492.33,9507.67,1,-492.33,0.0
1,closed,USDJPY,buy,100000,2023-03-16,133.373,2023-04-03,133.367,0,0,{},-4.00,-7.0,-11.0,-503.33,9496.67,0,-492.33,-11.0
2,closed,USDJPY,sell,100000,2023-04-17,133.802,2023-07-12,140.376,0,0,{},-4382.67,-7.0,-4389.67,-4893.0,5107.0,2,-492.33,-4400.67
3,closed,USDJPY,buy,100000,2023-07-13,138.456,2023-07-31,140.918,0,0,{},1641.33,-7.0,1634.33,-3258.67,6741.33,0,-492.33,-2766.34
4,closed,USDJPY,sell,100000,2023-08-03,143.324,2023-11-21,148.323,0,0,{},-3332.67,-7.0,-3339.67,-6598.34,3401.66,1,-492.33,-6106.01
5,closed,USDJPY,buy,100000,2023-11-22,148.372,2024-01-05,144.604,0,0,{},-2512.00,-7.0,-2519.0,-9117.34,882.66,4,-492.33,-8625.01
6,closed,USDJPY,sell,100000,2024-01-11,145.74,2024-03-07,149.306,0,0,{},-2377.33,-7.0,-2384.33,-11501.67,-1501.67,3,-492.33,-11009.34
7,closed,USDJPY,buy,100000,2024-03-08,147.96,2024-03-20,150.809,0,0,{},1899.33,-7.0,1892.33,-9609.34,390.66,2,-492.33,-9117.01
8,closed,USDJPY,sell,100000,2024-04-11,153.079,2024-06-05,154.849,0,0,{},-1180.00,-7.0,-1187.0,-10796.34,-796.34,2,-492.33,-10304.01
9,closed,USDJPY,sell,100000,2024-06-20,158.08,2024-07-18,156.165,0,0,{},1276.67,-7.0,1269.67,-9526.67,473.33,3,-492.33,-9034.34


ValueError: Plotly Express cannot process wide-form data with columns of different type.